In [1]:
import numpy as np
import pandas as pd
import nba_on_court as noc
from nba_api.stats.endpoints import CommonAllPlayers
from typing import Tuple

pd.set_option('display.max_columns', 500)

# Preprocessing Raw Data

Code for preprocessing the raw combined datasets. See `nba_on_court.left_join_nbastats` for combining NBA (nbastats) and Play-by-Play (pbpstats) datasets.

I've tried to include comments explaining some of my code whose purpose isn't obvious. However, the code may not be readable for anyone that isn't me.

"Stint" here deviates from traditional literature. It means any period of time where the possession of the ball remains constant. Traditional literature would refer to this as a "possession".
A stint ends when either a turnover occurs (including fouls), or when the score changes.

Data Assumptions
1. The only events that need to be considered are ones with a non-empty `SCOREMARGIN`, ones flagged as a turnover, or rebounds. These events mark the end of stints.
    - Event is a turnover iff `TURNOVERS == 1`.
    - Event is a rebound iff `EVENTMSGTYPE == 4`.
2. In each of the above events, the feature `PLAYER1_ID` corresponds to the player possessing the ball.
    - **Exception**: This may sometimes correspond to the team with the ball.
    - Empirically validated: https://github.com/rd11490/NBA_Tutorials/tree/master/analyze_play_by_play
    - This assumption is sometimes violated when there's a nonsensical value in `PLAYER1_ID`. 
        - These stints are very rare (<300), and are dropped from the data.
3. All stints have the same sample weight.
    - In traditional literature, sample weight usually corresponds to time elapsed.
    - Possessions generally all last around the same time.
4. A stint whose Plus-Minus is +-1 is a free-throw.
5. Free-throws do not constitute individual stints. Related free-throws are combined into one stint.
    - The "players on court" are the ones present in the preceeding foul.
    - **Minor Assumption**: All free-throws of length 1 are immediately preceeded by a foul with a made shot (counted as separate stints in the data)
        - I always merged the preceeding foul in this case, regardless of the sign of the score.
2. A game averages approximately 100 stints.
    - This dataset has its average at ~130.

In [2]:
# constants used for readable indexing

AWAY_LIST = ['AWAY_PLAYER1', 'AWAY_PLAYER2', 'AWAY_PLAYER3', 'AWAY_PLAYER4', 'AWAY_PLAYER5']
HOME_LIST = ['HOME_PLAYER1', 'HOME_PLAYER2', 'HOME_PLAYER3', 'HOME_PLAYER4', 'HOME_PLAYER5']
PLAYERS = AWAY_LIST + HOME_LIST

BASE_FEATURES  = [
    'GAME_ID',
    'SCOREMARGIN',
    'TURNOVERS', 
    'EVENTMSGTYPE',
    'PLAYER1_ID',
    'DESCRIPTION_PBP',
    'PERIOD',
    'URL'
] + PLAYERS

In [6]:
def preprocess_season(season: pd.DataFrame) -> pd.DataFrame:
    '''
    Given a full season of raw data, preprocess the data
    and return the resulting DataFrame. Note that this
    is essentially `preprocess_game` but for an entire season.

    This will return a DataFrame with the following features:
      `GAME_ID`: The game that the stint belongs to.
      `SCOREMARGIN`: The score margin of the stint. Note that positive means in favor for home, negative in favor for away.
      `TURNOVERS`: Indicator of whether the stint resulted in a turnover.
      `BALL_ID`: The player who possessed the ball during the stint.
      `PM`: The Plus-Minus of the stint. See SCOREMARGIN for meaning of the sign.
      `[AWAY|HOME]_PLAYER[NUM]`: Player ID of an Away/Home player.
    '''
    games = pd.unique(season['GAME_ID'])
    list_of_game_data = []

    for game in games:
        current = season[season['GAME_ID'] == game].reset_index(drop=True)
        current = noc.players_on_court(current)
        list_of_game_data.append(preprocess_game(current))

    return pd.concat(list_of_game_data, ignore_index=True)

FREETHROW_FEATURES = PLAYERS + ['BALL_ID']

def preprocess_game(game_data: pd.DataFrame) -> pd.DataFrame:
    '''
    Given the full data of a game (including players on court), 
    preprocess the game data and return the result.

    This will return a DataFrame with the following features:
      `GAME_ID`: The game that the stint belongs to.
      `SCOREMARGIN`: The score margin of the stint. Note that positive means in favor for home, negative in favor for away.
      `TURNOVERS`: Indicator of whether the stint resulted in a turnover.
      `BALL_ID`: (Presumed to be) the player who possessed the ball during the stint.
      `PM`: The Plus-Minus of the stint. See SCOREMARGIN for meaning of the sign.
      `[AWAY|HOME]_PLAYER[NUM]`: Player ID of an Away/Home player.
    '''
    # extract features we care about
    subset = game_data[BASE_FEATURES]
    subset.rename(columns={'PLAYER1_ID': 'BALL_ID'}, inplace=True)

    # have to preprocess score stints differently from stints that end in turnovers/rebounds
    # because otherwise the PM can't be calculated
    scores = subset[~subset['SCOREMARGIN'].isna()]
    scores['SCOREMARGIN'] = pd.to_numeric(scores['SCOREMARGIN'].replace('TIE', 0))
    scores['TURNOVERS'].fillna(0, inplace=True)
    scores['PM'] = scores['SCOREMARGIN'].diff().replace(np.nan, 0)
    scores.at[scores.index[0], 'PM'] = np.int64(scores.at[scores.index[0], 'SCOREMARGIN']) # manually add in the first PM
    score_stints = scores[(~scores['BALL_ID'].isna()) & (scores['BALL_ID'] != 0)]

    # now that we've identified the score stints, 
    # we can fill in SCOREMARGIN for everything else 
    # (this is needed for the propensity scores)
    subset['SCOREMARGIN'].ffill(inplace=True)
    subset['SCOREMARGIN'].fillna(0, inplace=True) # for events at the start

    # turnovers and rebounds are processed in the same exact way
    turnovers_and_rebounds = subset[(subset['TURNOVERS'] == 1) | (subset['EVENTMSGTYPE'] == 4)]
    turnovers_and_rebounds['SCOREMARGIN'] = pd.to_numeric(turnovers_and_rebounds['SCOREMARGIN'].replace('TIE', 0))
    turnovers_and_rebounds['PM'] = np.zeros(len(turnovers_and_rebounds), dtype=np.int64)
    turnover_rebound_stints = turnovers_and_rebounds[((~turnovers_and_rebounds['BALL_ID'].isna()) & (turnovers_and_rebounds['BALL_ID'] != 0))]

    stints = pd.concat([score_stints, turnover_rebound_stints]).sort_index().reset_index(drop=True)

    # deal with free throws
    # unfortunately i can't think of a vectorized solution, so we have to iterate over the rows
    rows_to_skip = []

    # there are 3 cases to consider.
    # case 1: fouled on a 2-point shot (and missed)
    #       - then 2 free-throws are awarded
    #       - we therefore must merge the following stint with the first free-throw.
    # case 2: fouled on a 3-point shot (and missed)
    #       - then 3 free-throws are awarded
    #       - we therefore must merge the following 2 stints with the first free-throw.
    # case 3: fouled on a made shot
    #       - then exactly 1 free-throw is awarded
    #       - this must be combined with the stint for the made shot, which is immediately preceeding the foul
    #
    # we can check which case it is by grabbing the 2 following rows, and determining if it's in the same free-throw run through the PM.
    #       - if the PM is 0 or equal to the first stint's PM, AND the stint is neither a turnover nor a rebound, then it is in the run
    #       - we can check for case 3 if none of the following turnovers are in the run
    
    for index, row in stints.iterrows():
        if np.abs(row['PM']) != 1 or index in rows_to_skip:
            continue
        i = stints.index.get_loc(index)

        foul_row = stints.iloc[i - 1]
        if foul_row['TURNOVERS'] == 0: # if the previous stint isn't a foul, treat the current stint as the foul.
            foul_row = row
        stints.loc[index, FREETHROW_FEATURES] = foul_row[FREETHROW_FEATURES]
        
        # check the next two rows if they're part of the same freethrow
        case3 = True
        for j in (1, 2):
            if i+j >= len(stints.index): # out of bounds
                break
            
            next_row = stints.iloc[i+j]
            if (row['PM'] != 0 and row['PM'] != next_row['PM']) or next_row['TURNOVERS'] == 1 or next_row['EVENTMSGTYPE'] == 4: # end of run
                break
            
            case3 = False
            stints.at[index, 'PM'] += next_row['PM']
            stints.at[index, 'SCOREMARGIN'] = next_row['SCOREMARGIN']
            rows_to_skip.append(stints.index[i+j])

        if case3:
            # case 3 is special because sometimes the foul can be the preceeding stint, or the 2nd-preceeding stint
            # if neither case holds, treat the current stint as the foul
            foul_row = stints.iloc[i - 1]
            if foul_row['TURNOVERS'] == 0:
                foul_row = stints.iloc[i - 2]
                if foul_row['TURNOVERS'] == 0:
                    foul_row = row
            stints.loc[index, FREETHROW_FEATURES] = foul_row[FREETHROW_FEATURES]

            j=1
            prev_row = stints.iloc[i-j]
            while prev_row['TURNOVERS'] == 1: # keep going back until we reach a score stint (in case the foul is reported first)
                j += 1
                prev_row = stints.iloc[i-j]

            stints.at[index, 'PM'] += prev_row['PM']
            rows_to_skip.append(stints.index[i-j])

    # finally, filter out the rows flagged for skipping
    stints.drop(rows_to_skip, inplace=True)
    
    return stints

In [7]:
raw_data = pd.read_csv('../../data/combined_data_2022.csv', index_col=0)
data = preprocess_season(raw_data)

data.head(50)

,GAME_ID,SCOREMARGIN,TURNOVERS,EVENTMSGTYPE,BALL_ID,DESCRIPTION_PBP,PERIOD,URL,AWAY_PLAYER1,AWAY_PLAYER2,AWAY_PLAYER3,AWAY_PLAYER4,AWAY_PLAYER5,HOME_PLAYER1,HOME_PLAYER2,HOME_PLAYER3,HOME_PLAYER4,HOME_PLAYER5,PM
0,42200101,0,0.0,4,203507,g. antetokounmpo rebound (off:0 def:1),1,https://videos.nba.com/nba/pbp/media/2023/04/1...,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,0.0
1,42200101,2,0.0,1,203507,g. antetokounmpo 2' driving finger roll layup ...,1,https://videos.nba.com/nba/pbp/media/2023/04/1...,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,2.0
2,42200101,0,0.0,1,1629639,herro 13' driving floating jump shot (2 pts) (...,1,https://videos.nba.com/nba/pbp/media/2023/04/1...,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,-2.0
3,42200101,2,0.0,1,1628960,allen 3' alley oop dunk (2 pts) (holiday 2 ast),1,https://videos.nba.com/nba/pbp/media/2023/04/1...,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,2.0
4,42200101,-1,0.0,1,1629639,herro 26' 3pt jump shot (5 pts) (vincent 1 ast),1,https://videos.nba.com/nba/pbp/media/2023/04/1...,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,-3.0
5,42200101,-1,1.0,5,1628960,allen step out of bounds turnover (p1.t1),1,https://videos.nba.com/nba/pbp/media/2023/04/1...,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,0.0
6,42200101,-1,1.0,5,1629216,durant step out of bounds turnover (p1.t1),1,https://videos.nba.com/nba/pbp/media/2023/04/1...,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,0.0
7,42200101,-1,0.0,4,1628389,adebayo rebound (off:0 def:2),1,https://videos.nba.com/nba/pbp/media/2023/05/1...,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,0.0
8,42200101,-3,0.0,1,202710,butler 1' running finger roll layup (2 pts) (a...,1,https://videos.nba.com/nba/pbp/media/2023/04/1...,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,-2.0
9,42200101,-2,0.0,3,203507,g. antetokounmpo free throw 2 of 2 (3 pts),1,NaN,1629622,1629639,1629216,202710,1628389,201572,203114,201950,1628960,203507,1.0


Sometimes, `BALL_ID` corresponds to the offensive team instead of the offensive player. In these cases, we need to replace the `BALL_ID` with one of the players in the team.

In [8]:
player_data = CommonAllPlayers(
    is_only_current_season=1,
    league_id='00',
    season='2022-23'
).get_data_frames()[0]

def check(row: pd.Series):
    return row['BALL_ID'] not in row[PLAYERS].values

# there's definitely a vectorized solution, i'm just not sure if you can edit rows in-place in DataFrame.apply
rows_to_drop = []

for index, row in data[data.apply(check, axis=1)].iterrows():
    team_roster: pd.DataFrame = player_data[player_data['TEAM_ID'] == row['BALL_ID']]
    away_plr                  = row['AWAY_PLAYER1']
    home_plr                  = row['HOME_PLAYER1']

    if away_plr in team_roster['PERSON_ID'].values:
        data.at[index, 'BALL_ID'] = away_plr
    elif home_plr in team_roster['PERSON_ID'].values:
        data.at[index, 'BALL_ID'] = home_plr
    else:
        rows_to_drop.append(index) # nonsensical team

data.drop(rows_to_drop, inplace=True)
data.reset_index(drop=True, inplace=True)


In [9]:
data[['SCOREMARGIN', 'TURNOVERS', 'PM']].head(50)

,SCOREMARGIN,TURNOVERS,PM
0,0,0.0,0.0
1,2,0.0,2.0
2,0,0.0,-2.0
3,2,0.0,2.0
4,-1,0.0,-3.0
5,-1,1.0,0.0
6,-1,1.0,0.0
7,-1,0.0,0.0
8,-3,0.0,-2.0
9,-2,0.0,1.0


In [10]:
data.to_csv('../../data/processed_data_2022.csv')

# Design Matrix

Using the processed data, create a design matrix out of it.

In [11]:
data = pd.read_csv('../../data/processed_data_2022.csv', index_col=0)

In [12]:
def _determine_coefficient_odrapm(row: pd.Series, player_id: int, offensive: bool) -> int:
    '''
    Determines whether a player's coefficient is -1, 0, 1 in a given stint for Offensive/Defensive RAPM model.

    This is meant to be used in the `get_design_matrix` function.
    '''
    away = row[AWAY_LIST].values
    home = row[HOME_LIST].values
    away_is_offense = row['BALL_ID'] in away                                # True if Away is offensive team
    away_is_offense = away_is_offense if offensive else not away_is_offense # Basically, change 'defense' case to 'offense' case

    if player_id in away and away_is_offense:
        return -1
    elif player_id in home and not away_is_offense:
        return 1
    else:
        return 0
    
def get_design_matrix_odrapm(game_data: pd.DataFrame, *, as_frame: bool = True) -> pd.DataFrame | Tuple[np.ndarray, np.ndarray]:
    '''
    Given a preprocessed game/season of data, return a design matrix for a regression model.
    This will not add the signed intercept, that needs to be added separately.

    Each row of the matrix will correspond to a stint.
    Even columns (0, 2, 4, ...) correspond to offensive indicators (i.e. filled iff player is on offense).\n
    Odd columns (1, 3, 5, ...) correspond to defensive indicators (i.e. filled iff player is on defense).\n
    Player `k` corresponds to columns `2k` (offense) and `2k+1` (defense).\n
    
    The matrix will be filled as follows:
      `-1`: Present on the Away team.
      `0`: Not present during the stint.
      `1`: Present on the Home team. 

    By default, the result is returned as a DataFrame. 
    Note that defensive indicators may have a `.1` at the end of the column feature. 
    
    If `as_frame` is False, then this will return a tuple of numpy arrays. 
    The first array is the design matrix itself.
    The second array is the list of players, where position `i` corresponds to columns `2i` and `2i + 1`.

    THIS CODE IS NOT WELL OPTIMIZED. THERE ARE DEFINITELY FASTER SOLUTIONS, 
    BUT DUE TO HOW RARELY THIS NEEDS TO BE RUN, I'M TOO LAZY TO MAKE THEM.
    THIS TAKES 50-70 MINUTES TO FINISH RUNNING.
    '''
    all_players = np.unique(game_data.filter(like='PLAYER').to_numpy())
    design_matrix = np.empty((len(game_data), 2 * len(all_players)))        # stints x 2 * players. to be filled

    # fill the design matrix with coefficients
    # for a future date: call 'apply' once, have _det_coeff return a 2-sized vector, and fill both columns simultaneously
    for i in range(len(all_players)):
        player = all_players[i]
        design_matrix[:, 2*i] = game_data.apply(_determine_coefficient_odrapm, axis=1, player_id=player, offensive=True).to_numpy()
        design_matrix[:, 2*i+1] = game_data.apply(_determine_coefficient_odrapm, axis=1, player_id=player, offensive=False).to_numpy()

    if as_frame:
        return pd.DataFrame(data=design_matrix, columns=np.repeat(all_players, 2))
    else:
        return design_matrix, all_players

In [13]:
matrix = get_design_matrix_odrapm(data)

And now add the signed intercept column

In [14]:
def _determine_coefficient(row: pd.Series) -> int:
    '''
    Determines the coefficient for the signed constant term in the ridge regression design matrix.

    1 if Home is offense, -1 otherwise.
    '''
    if row['BALL_ID'] in row[HOME_LIST].values: # if home is offense
        return 1
    else:
        return -1

In [15]:
coefs = data.apply(_determine_coefficient, axis=1)
matrix.insert(loc=0, column='0', value=coefs)


In [17]:
matrix.columns

Index([    '0',    2544,    2544,    2617,    2617,  101108,  101108,  200768,
        200768,  200782,
       ...
       1631128, 1631128, 1631169, 1631169, 1631212, 1631212, 1631223, 1631223,
       1631254, 1631254],
      dtype='object', length=435)

In [16]:
matrix.to_csv('../../design_matrices/odrapm_design.csv.gz', compression='gzip')